# Progetto Carburante

## Obiettivo (rileggere ogni volta, per ricordarsi l'obiettivo)
Al fine di esercitarmi con Python e con le tecniche di data analysis, voglio utilizzare i dati del ministero e dell'ISTAT.

## Orizzonti e limiti
Voglio fare un'analisi su 
* orizzonte temporale di qualche anno (inizio da 2017 - 2018)
* granularità geografica sulle PROVINCE


# Dati di partenza
## Anagrafica pompe
I dati csv delle pompe hanno questo formato:

| idImpianto | Gestore | Bandiera | Tipo Impianto | Nome Impianto | Indirizzo | Comune | Provincia | Latitudine | Longitudine |
|------------|---------|---------|---------|---------|---------|---------|---------|---------|---------|

Esiste un file diverso per ogni giorno: l'anagrafica è tempo variante per:
* creazione nuove pompe
* eliminazione vecchie pompe
* variazioni pompe (può cambiare bandiera, per esempio)


### Dati di analisi

Ai miei fini, voglio usare
* **idImpianto**: id proveniente dall'excel, va relazionato coi prezzi del giorno (csv da associare)
* **Bandiera**: "marca" della pompa (Esso, Agip, Pompe Bianche). Mi interessa per fare analisi fra marche diverse
* **Tipo Impianto**: strada statale o autostrada. 
* **Latitudine/Longitudine**: è la vera chiave forte, che mi permette di analizzare le variazioni anagrafiche.
* **Provincia**: codice provincia, sarà la base per la ripartizione geografica.
* **Comune**: voglio usarlo per verifiche/raccordi anagrafici (in alcuni dati manca la provincia ma posso cercare di risalire tramite il comune)


### Pulizia dei dati
Alcune pompe non hanno la provincia!


# Data Warehouse
## DimImpianto
A partire dai dati anagrafici di partenza voglio creare una DimImpianto:
* **id unificato**: chiave forte dell'impianto (sarà 1:1 con la coppia (lat, long), di fatto)
* **Regione**: dovrò riconciliare con le province
* **Area**: Nord / Centro / Sud / Isole
* **Data inizio**: data prima presenza in anagrafica
* **Data fine** data ultima rilevazione
* **Latitudine/Longitudine**: come da input
* **Tipo Impianto**: come da input
* **Provincia**: come da input
